In [2]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
import pandas as pd

# Load your dataset into a pandas DataFrame
df = pd.read_csv('walmart_products.csv')

In [3]:
df

,user_id,product_id,product_name,rating
0,user1,1,Product A,4.5
1,user1,2,Product B,3.0
2,user1,3,Product C,2.5
3,user1,4,Product D,3.0
4,user1,5,Product E,4.0
...,...,...,...,...
351,user50,31,Product EE,2.0
352,user50,13,Product M,3.0
353,user50,26,Product Z,4.5
354,user50,6,Product F,2.5


In [4]:
# Create a Surprise Reader
reader = Reader(rating_scale=(1, 5))

# Load the dataset
data = Dataset.load_from_df(df[['user_id', 'product_id', 'rating']], reader)

# Split the dataset into train and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Create an SVD model
model = SVD()

# Train the model on the trainset
model.fit(trainset)

# Predict ratings on the testset
predictions = model.test(testset)

# Get predicted ratings for all users and products
predicted_ratings = {}
for uid, iid, true_r, est, _ in predictions:
    if uid not in predicted_ratings:
        predicted_ratings[uid] = {}
    predicted_ratings[uid][iid] = est

In [19]:
def get_recommendations_with_names(user_id, min_num_recommendations=4):
    if user_id in predicted_ratings:
        user_predictions = predicted_ratings[user_id]
        sorted_predictions = sorted(user_predictions.items(), key=lambda x: x[1], reverse=True)
        recommended_products = [product_id for product_id, _ in sorted_predictions]
        
        # Get the product names corresponding to recommended product IDs from the dataset
        recommended_product_names = [df.loc[(df['user_id'] == user_id) & (df['product_id'] == product_id), 'product_name'].values[0] for product_id in recommended_products]
        
        # Ensure at least min_num_recommendations recommendations are provided
        while len(recommended_products) < min_num_recommendations:
            additional_recommendation = recommended_products[len(recommended_products) - 1]
            recommended_products.append(additional_recommendation)
            recommended_product_names.append(df.loc[(df['user_id'] == user_id) & (df['product_id'] == additional_recommendation), 'product_name'].values[0])
        
        return recommended_products[:min_num_recommendations], recommended_product_names[:min_num_recommendations]
    else:
        return [], []

# Get recommendations for a particular user
user_id = 'user19'  # Replace with the desired user ID
min_num_recommendations = 4
recommendations_ids, recommendations_names = get_recommendations_with_names(user_id, min_num_recommendations)

if recommendations_ids:
    print("Recommended products for user", user_id, ":")
    for product_id, product_name in zip(recommendations_ids, recommendations_names):
        print("Product ID:", product_id, "- Product Name:", product_name)
else:
    print("No recommendations available for user", user_id)

Recommended products for user user19 :
Product ID: 34 - Product Name: Product HH
Product ID: 3 - Product Name: Product C
Product ID: 18 - Product Name: Product R
Product ID: 35 - Product Name: Product II
